In [1]:
import random
import copy
import time
import asyncio
import websockets
import numpy as np

from first import get_numbers

### helper functions

In [2]:
def generate_random_list(t):
    """Generate list of random digits of the length t
    """
    return [random.randint(0,9) for i in range(t)]

def generate_noised_images_list(width,height,p,ethalon_matrices_list):
    """Add a noise to the ethalon images.
    Args:
        width - int
        height - int
        p - (float) probability of a noise
        ethalon_matrices_list - a list of a ethalon matrices
    Returns:
        noised_images - a list of a noised matrices, length tha same as in the 'ethalon_matrices_list'
    """
    noised_images = list()
    for matrix in ethalon_matrices_list:
        noise = np.random.choice(a=[0, 1], size=(w,h), p=[p, 1-p])
        noised_image = np.bitwise_xor(matrix,noise)
        noised_images.append(noised_image)
    return noised_images

def calculate_prob(x_noised,ethalon_matrices,target_index,histogram,p_):
    """Due to the seminar notation, this fuction try to calculate p(x_noised,true_matrices_list[target_index])
    Args:
        x_noised - ndarray 100*120
        true_matrices_list - the list of 10 ndarray 100*120
        target_index - int in [0,..,9]
        p - float in [0,1]
    
    """
    xor_target = x_noised^ethalon_matrices[target_index]
    result_sum = 0
    for index_matrix,true_matrix in enumerate(ethalon_matrices):
        xor_matrix = x_noised^true_matrix
        # the equation from the seminar
        result_sum += histogram[index_matrix]*(1-p_/p_)**((xor_matrix - xor_target).sum())
        #print(result_sum)
    #print((1-p_/p_))
    return histogram[target_index]/result_sum

def calc_prob_list(digit_index):
    """Calculate a list of possibilities that digit_index corresponds to [0,..,9] 
    """
    possible_sum = list()
    for i in range(10):
        prob = calculate_prob(noised_images[digit_index],int_to_image_list,i,histogram,p)
        possible_sum.append(prob)
    return possible_sum

def calc_partial_sum(previous_probs_list_,current_probs_list_,t):
    """This function is convolution when dimension = 2.
    Args:
        previous_probs_list - a list of length = 9*t +1,where at position i, the probability, that the
            first (t-1) digits' sum = i
        current_probs_list_ -  a list of length = 10, ,where at position i, the probability, that the
            t-th digit equals to i
        t - int, denotes current position , we calculate sums for
    """
    predictions = list(np.zeros((9*(t+1) +1)))
    for current_value,current_prob in enumerate(current_probs_list_):
        for prev_value,prev_prob in enumerate(previous_probs_list_):
            predictions[current_value + prev_value] += current_prob*prev_prob
            #print(current_prob)
    return predictions

def sum_digits(number):
    """Calcuate the sum of digits in 'number'
    """
    digits_sum = 0
    while number:
        digits_sum += number % 10
        number //= 10
    return digits_sum

### generate random sequence

In [3]:
t = 21
random_int_list = generate_random_list(t)
# distribution of digits in random_int_list(uniform)
histogram = [1/t for i in range(t)]
print(random_int_list)

[0, 8, 9, 3, 5, 3, 7, 0, 5, 0, 9, 0, 0, 6, 3, 0, 4, 7, 6, 7, 2]


### generating ethalon images

In [4]:
# default width = 3, height = 5
width_scale = 100
height_scale = 100
# list, at which on a position i, array which represents i
int_to_image_list = asyncio.get_event_loop().run_until_complete(get_numbers(1,1))
for i in range(len(int_to_image_list)):
    int_to_image_list[i] = np.repeat(
        np.repeat(int_to_image_list[i],width_scale,axis=0),height_scale,axis=1)
# random_int_list - list of random digits, e.g. [4,5,6,8]
ethalon_matrices_list = [int_to_image_list[random_int] for random_int in random_int_list]
print("Each image has a shape : ",ethalon_matrices_list[0].shape)

Each image has a shape :  (500, 300)


### noising images

In [5]:
w, h = 5*height_scale,3*width_scale
p = 0.9
# adding noise to the 'ethalon_matrices_list'
noised_images = generate_noised_images_list(w,h,p,ethalon_matrices_list)
print("noised_images.shape :",noised_images[0].shape)

noised_images.shape : (500, 300)


### calculate the most pobable sum

In [6]:
start_time = time.time()
# up to 't' numbers calculate sum
t = 21
for current_t in range(0,t):
    if current_t == 0 : 
        previous_probs_list = calc_prob_list(0)
    else:
        next_probs_list = calc_prob_list(current_t)
        previous_probs_list = calc_partial_sum(previous_probs_list,next_probs_list,current_t)
end_time = time.time()
print("elapsed time : ",end_time - start_time)

c:\install\python\python36\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in power


elapsed time :  1.1021804809570312


In [7]:
print("predicted sum : ",np.array(previous_probs_list).argmax())
print("real sum : ",np.sum(np.array(random_int_list[:t])).sum())

predicted sum :  84
real sum :  84


### Check is sum is divided by 3

In [10]:
prob_is_divided = 0
prob_is_not_divided = 0
for digits_sum,prob in enumerate(previous_probs_list):
    if digits_sum % 3 == 0:
        prob_is_divided += prob
    else :
        prob_is_not_divided += prob

if prob_is_divided > prob_is_not_divided:
    print("Our sum is divided by 3 without a residual")
else :
    print("Our sum is divided by 3 with a residual")

Our sum is divided by 3 without a residual
